#Import neccassary libraries

In [18]:
import pandas as pd
import numpy as np

#Read the text from the input files d1,d2,d3,d4

In [19]:
data_dir = "/content/"
data_files = ["d1.txt", "d2.txt", "d3.txt","d4.txt"]

data = pd.Series()
for file in data_files:
    f = open(data_dir + file, 'r')
    file_contents = f.read()
    data[file[:-4]] = file_contents
    f.close()
    print(data[file[:-4]])

User selected Wedding gown.

User ordered on-line rose flowers.

User searched diamond ring.

User selected white wedding gown, online flowers, 3 carat diamond ring.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


#Count the words in each doc

In [20]:
from collections import Counter

bag_of_words = (
    data.
    str.lower().                  # converting  each letter to lowercase
    str.replace("[^\w\s]", " ").  # replacing non-alphanumeric characters by whitespace
    str.split()                   # spliting on whitespace
).apply(Counter)

bag_of_words

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


d1    {'user': 1, 'selected': 1, 'wedding': 1, 'gown...
d2    {'user': 1, 'ordered': 1, 'on': 1, 'line': 1, ...
d3    {'user': 1, 'searched': 1, 'diamond': 1, 'ring...
d4    {'user': 1, 'selected': 1, 'white': 1, 'weddin...
dtype: object

#determining TF

In [21]:
tf = pd.DataFrame(list(bag_of_words))
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0


#Replace NA with 0

In [22]:
tf = tf.fillna(0)
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


#Document frequencies df

In [23]:
df = (tf > 0).sum(axis=0)
df

user        4
selected    2
wedding     2
gown        2
ordered     1
on          1
line        1
rose        1
flowers     2
searched    1
diamond     2
ring        2
white       1
online      1
3           1
carat       1
dtype: int64

#determine IDF

In [24]:
idf = np.log(len(tf) / df)
idf

user        0.000000
selected    0.693147
wedding     0.693147
gown        0.693147
ordered     1.386294
on          1.386294
line        1.386294
rose        1.386294
flowers     0.693147
searched    1.386294
diamond     0.693147
ring        0.693147
white       1.386294
online      1.386294
3           1.386294
carat       1.386294
dtype: float64

#Calculating TF-IDF

In [25]:
tf_idf = tf * idf
tf_idf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,1.386294,1.386294,1.386294,1.386294,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000
3,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.693147,1.386294,1.386294,1.386294,1.386294


#Cosine Similarity

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) 
vec.fit(data) #find the vocabulary
tf_idf_sparse = vec.transform(data)

#Calculate the numerator

In [27]:
a = tf_idf_sparse[0, :]
b = tf_idf_sparse[2, :]
dot = a.multiply(b).sum()

#calculate the denominator

In [28]:
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(b.multiply(b).sum())

#Determine their ratio to find the cosine similarity

In [29]:
dot / (a_len * b_len)

0.11744989547797277

#numerators

In [30]:
a = tf_idf_sparse[0, :]
B = tf_idf_sparse
dot = a.multiply(B).sum(axis=1)
dot

matrix([[7.8477822],
        [1.       ],
        [1.       ],
        [7.8477822]])

#calculate the denominators

In [31]:
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(B.multiply(B).sum(axis=1))
print(a_len)
b_len

2.8013893332208726


matrix([[2.80138933],
        [4.23925403],
        [3.03930227],
        [5.07070753]])

#determine their ratio to find the cosine similarity

In [32]:
dot / (a_len * b_len)

matrix([[1.        ],
        [0.08420485],
        [0.1174499 ],
        [0.55246518]])

#Result

In [33]:
cos_similarities = pd.DataFrame(dot / (a_len * b_len))[0]
most_similar = cos_similarities.sort_values(ascending=False)
most_similar

0    1.000000
3    0.552465
2    0.117450
1    0.084205
Name: 0, dtype: float64